### Set Up

In [ ]:
from functools import partial
from pathlib import Path
import os, io, cv2, joblib, random, warnings, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches
from collections import Counter
import seaborn as sns
from datetime import datetime
from PIL import Image
from sklearn.utils import resample
from imblearn.over_sampling import RandomOverSampler
import pickle
from skimage.feature import hog
from sklearn.decomposition import PCA
import zipfile
from skimage import io, color, img_as_ubyte, data, exposure, img_as_float
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cluster import MiniBatchKMeans
import sklearn.svm as svm
from sklearn.preprocessing import LabelEncoder

import keras_tuner
import keras
import matplotlib
from scikeras.wrappers import KerasClassifier
import tensorflow.keras as K
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Lambda, Dense, GlobalAveragePooling2D, Flatten, Dropout, Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization, Rescaling, AveragePooling2D, Input, Add, Activation
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, History, ModelCheckpoint

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess

from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input  as xception_preprocess

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input  as vgg16_preprocess

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preproces

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inceptionv3_preprocess


from kerastuner import HyperModel, RandomSearch

from matplotlib import rc
import matplotlib.animation as animation

rc('animation', html='jshtml')

plt.rcParams['animation.embed_limit'] = 50.0

import logging
logging.disable(logging.WARNING)

%matplotlib inline
%load_ext autotime

In [ ]:
# Set random seed

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

time: 985 µs (started: 2024-04-01 10:44:08 +00:00)


In [ ]:
# Disable warning

warnings.filterwarnings("ignore", category=UserWarning)
tf.get_logger().setLevel('ERROR')

time: 359 µs (started: 2024-04-01 10:44:08 +00:00)


In [ ]:
# Define path for saved models, and video for evalution

VIDEO_PATH = f"{GOOGLE_DRIVE_PATH}/Video"
MODEL_PATH = f"{GOOGLE_DRIVE_PATH}/Models"

### Tools

In [ ]:
def _get_timestamp(format="%Y%m%d-%H%M%S"):
    """
      Get current timestamp
    """

    current_date = datetime.now()
    timestamp = current_date.strftime(format)

    return timestamp

time: 530 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _get_data(path, split_size=None):
    """
      Import data with data splitting for validation set
    """

    # Import data
    X, y = _import_data(path)

    # If split size defined
    if split_size:
        X_a, X_b, y_a, y_b = train_test_split(X, y,
                                              test_size=split_size,
                                              random_state=42,
                                              shuffle=True,
                                              stratify=y)

        return X_a, X_b, y_a, y_b

    return X, y

time: 708 µs (started: 2024-03-29 12:34:04 +00:00)


In [ ]:
def _import_data(path, n=None, filenames=None):
    """
      Import images and labels
    """

    images = []
    labels = []

    # If files not exists, unzip input data
    if not Path(path).exists():
        _unzip_data()

    if not filenames:
        # Get index of images and labels
        image_path = sorted(os.listdir(f'{path}/images'))
        label_path = sorted(os.listdir(f'{path}/labels'))

        # Zip to retain index
        zipped_lists = list(zip(image_path, label_path))

        # Random shuffle to get randomly picture for evaluation
        random.shuffle(zipped_lists)

        image_path, label_path = zip(*zipped_lists)

    else:
        image_path = [f'image_{fn}.jpeg' for fn in filenames]
        label_path = [f'image_{fn}.txt' for fn in filenames]

    i = 0
    for img, label in zip(image_path, label_path):
        # Get only n pictures
        if n is not None:
            if i >= n:
                break

        # Get pixel of images, and read labels inside text files
        img_val = io.imread(os.path.join(f'{path}/images', img))
        label_val = open(os.path.join(f'{path}/labels', label)).read()

        images.append(img_val)
        labels.append(int(label_val))

        i+=1

    return images, labels

time: 752 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _unzip_data(filename='CV2024_CW_Dataset.zip'):
    """
      Unzip input data
      Code obtained from Lab tutorial 06
    """

    zip_path = os.path.join(GOOGLE_DRIVE_PATH, f'CW_Dataset/{filename}')

    !cp '{zip_path}' .

    !yes|unzip -q '{filename}'

    !rm '{filename}'

In [ ]:
def _resize(images, size):
    """
      Resize images to desire size
    """

    return [cv2.resize(image, size, interpolation=cv2.INTER_LINEAR) for image in images]

time: 462 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _convert_to_array(X, y):
    """
      Convert images to array
    """

    X_array = np.array(X)
    y_array = np.array(y)

    return X_array, y_array

time: 400 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _convert_to_float(images):
    """
      Convert to float datatype
    """

    return img_as_float(images)

time: 373 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _convert_to_int(images):
    """
      Convert to int datatype
    """

    return img_as_ubyte(images)

time: 674 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _bgr2gray(images):
    """
    Convert BGR to Gray scale
    """

    return cv2.cvtColor(images, cv2.COLOR_BGR2GRAY)

time: 377 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _rgb2gray(images):
    """
    Convert RGB to Gray scale
    """

    images_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in images]
    return np.array(images_gray)

time: 464 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _normalize(images):
    """
      Normalise image to range between 0 and 1
    """

    return images / 255.0

time: 356 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _equalizehist(images):
    """
      Histogram Equalization
      Code obtained from https://stackoverflow.com/questions/31998428/opencv-python-equalizehist-colored-image
    """

    equalized = []

    for img in images:

        # Covert BGR to YCrCb
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

        # Equalize on Y channel
        img[:, :, 0] = cv2.equalizeHist(img[:, :, 0])

        # Convert back to BGR
        img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)

        equalized.append(img)

    return np.array(equalized)

time: 551 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _shuffle(X, y):
    """
      Shuffle records
    """

    X_shuffle, y_shuffle = shuffle(X, y)

    return X_shuffle, y_shuffle

time: 785 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _randomoversampler(X, y, input_size=(64,64)):
    """
      Oversampling minority class with bootstaping
    """

    # Upsample data with bootstraping
    ros = RandomOverSampler(random_state=42)

    X_flat = X.reshape(X.shape[0],-1)
    X_upsampled, y_upsampled = ros.fit_resample(X_flat, y)
    X_upsampled = X_upsampled.reshape(-1, input_size[0], input_size[1], 3)

    return X_upsampled, y_upsampled

time: 572 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _augmentation(rotation_range=10,
                  horizontal_flip=True,
                  fill_mode='nearest',
                  zoom_range=0.1):
    """
      Data augmentation
    """

    # Define an image data generator with specified augmentation parameters
    generator = ImageDataGenerator(rotation_range=rotation_range,
                                   horizontal_flip=horizontal_flip,
                                   fill_mode=fill_mode,
                                   zoom_range=zoom_range)

    return generator

time: 451 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _gen_augmented_data(X, y, batch_size=32):
    """
      Data augmentation
    """

    augmented_data = []
    augmented_labels = []

    # Create a generator for augmented data
    generator = _augmentation().flow(_convert_to_float(X), y, batch_size=batch_size)

    # Iterate through the generator and collect augmented data until the length of augmented data matches the original data
    for X_batch, y_batch in generator:
        augmented_data.extend(X_batch)
        augmented_labels.extend(y_batch)

        if len(augmented_data) >= len(X):
            break

    # Convert lists to arrays
    X_augmented = np.array(_convert_to_int(augmented_data))
    y_augmented = np.array(augmented_labels)

    return X_augmented, y_augmented

time: 637 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _gridsearchcv(classifier, X, y, params, folds=5):
    """
    Perform Gridsearch for hyperparameter tuning
    """

    # Use Stratify n-folds cross validation
    cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

    # Initialize GridSearchCV object
    grid = GridSearchCV(classifier, params, verbose=1, refit=True, n_jobs=-1, scoring="f1_macro", cv=cv)

    # Perform grid search
    grid.fit(X, y)

    print(f"Best params : {grid.best_params_}")
    print(f"Best scores : {grid.best_score_}")

    return grid

time: 624 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _gen_performance_metrics(y, y_pred):
    """
    Get accuracy and classification report
    """

    # Print classification report for evaluation
    accuracy = accuracy_score(y, y_pred)
    print("Accuracy:", accuracy)
    print(classification_report(y, y_pred))

time: 423 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _confusion_matrix(y, y_pred):
    """
      Visualize confusion matrix
    """

    # Calculate and visualize confusion matric for evaluation
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y, y_pred), annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.show()

time: 482 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _history_plot(history):
    """
      Plot training history (accuracy and loss)
    """

    # Create subplots
    fig, axs = plt.subplots(2, 1, figsize=(10, 8))

    history_log_df = pd.DataFrame(history)

    # Plot accuracy
    sns.lineplot(data=history_log_df[['accuracy','val_accuracy']], ax=axs[0])
    axs[0].set_title('Accuracy')

    # Plot loss
    sns.lineplot(data=history_log_df[['loss','val_loss']], ax=axs[1])
    axs[1].set_title('Loss')

time: 596 µs (started: 2024-03-29 11:37:13 +00:00)


In [ ]:
def _evaluate(model, history, X_train, y_train, X_validate, y_validate):
    """
      Evaluation on train and validation set
    """

    # Evaluate train set
    y_pred_train = model.predict(X_train)
    _gen_performance_metrics(np.argmax(y_train, axis=1), np.argmax(y_pred_train, axis=1))

    # Evaluate validation set
    y_pred_validate = model.predict(X_validate)
    _gen_performance_metrics(np.argmax(y_validate, axis=1), np.argmax(y_pred_validate, axis=1))

    # Visualize history loss and accuracy
    _history_plot(history)

In [ ]:
def _plot_images_with_labels(X_test, y_test, y_pred_test, label_true, label_pred):
    """
      Plot 4 images with specify true label and predicted labels, to perform error analysis
    """

    count = 0
    plt.figure(figsize=(12, 8))
    for i in range(len(y_test)):
        if y_test[i] == label_true and y_pred_test[i] == label_pred:
            plt.subplot(1, 4, count + 1)
            plt.imshow(X_test[i])
            plt.title(f'True: {y_test[i]}, Predicted: {y_pred_test[i]}')
            plt.axis('off')
            count += 1
            if count == 4:
                break
    plt.tight_layout()
    plt.show()

In [ ]:
def _plot_images(X, y_true, y_pred):
    """
      Plot images with true and predicted labels
    """

    num_images = len(y_true)
    plt.figure(figsize=(12, 8))
    for i in range(num_images):
        plt.subplot(1, num_images, i+1)
        plt.imshow(X[i])
        plt.title(f'True: {y_true[i]}, Predicted: {y_pred[i]}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def _video_detection(path, detector, preprocess, model):
    """
      Detect objects in a video using a given detector model
      Code derived from Lab 09
    """

    # Load the video file
    video_path = f"{VIDEO_PATH}/{path}"
    cap = cv2.VideoCapture(video_path)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create an array to store processed frames
    video = np.empty((frameCount//10, frameHeight, frameWidth, 3), np.dtype('uint8'))
    print('video shape =', video.shape)
    print(f'Frame count : {frameCount}')

    fc = 0
    ret = True
    i = 0

    # Process each frame of the video
    while cap.isOpened() and ret and i < frameCount//10:
        ret, frame = cap.read()

        # Process every 10th frame
        if fc % 10 == 0:

            # Perform face, and face mask detection
            frame = detector(frame, preprocess, model)

            # Convert back to RGB
            video[i] = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            i += 1

        fc += 1

    cap.release()

    return video, i

In [ ]:
def face_detector(path, detector, preprocess, model):
    """
      Detect faces in frames' video using a given face detector model
      Code derived from Lab 09
    """

    # Process video frames to detect faces
    video, frames = _video_detection(path, detector, preprocess, model)
    fig, ax = plt.subplots(figsize=(5, 3))

    # Create animation of detected faces
    def frame(i):
        ax.clear()
        ax.axis('off')
        fig.tight_layout()
        plot=ax.imshow(video[i, :, :, :])
        return plot

    anim = animation.FuncAnimation(fig, frame, frames=frames)
    plt.close()

    return anim

In [ ]:
def yunet_face_detector(frame, preprocess, model, lw=10):
    """
      Detect faces in a video frame using Yunet face detection model
      Code derived from Lab 08, https://docs.opencv.org/4.x/d0/dd4/tutorial_dnn_face.html
      Model obtained from https://github.com/opencv/opencv_zoo/tree/main/models/face_detection_yunet
    """

    # Create Yunet face detector
    yn = cv2.FaceDetectorYN.create(f'{GOOGLE_DRIVE_PATH}/Code/face_detection_yunet_2023mar.onnx',  '', (0, 0))
    h, w, _ = frame.shape
    yn.setInputSize((w, h))

    # Detect faces
    _, faces = yn.detect(frame)

    if faces is not None:
        for face in faces:

            # Extract detected face
            detected_face = frame[int(face[1]):int(face[1])+int(face[3]), int(face[0]):int(face[0])+int(face[2])]

            # Predict label for detected face
            color, text = _predict_detected_face(detected_face, preprocess, model)

            # Draw rectangle around detected face
            cv2.rectangle(frame, [int(face[0]), int(face[1]), int(face[2]), int(face[3])], color, lw)

            # Add label to the detected face
            cv2.putText(frame, text, (int(face[0]), int(face[1])-10), cv2.FONT_HERSHEY_TRIPLEX, 1.5, color, 3)

    return frame

time: 1.81 ms (started: 2024-04-04 11:05:34 +00:00)


In [ ]:
def _predict_detected_face(detected_face, preprocess, model):
      """
        Predict label and confidence score for a detected face
      """

      # Predict labels of face mask classes with confidence score
      _, pred, confidence_scores = preprocess._evaluate_frame([detected_face], None, model)
      score = f"{round(confidence_scores[0]*100,1)} %"

      # Define color and text for each class
      if pred[0] == 0:
        color = (0, 0, 255)
        text = f'No Mask, {score}'

      elif pred[0] == 1:
        color = (0, 255, 0)
        text = f'Mask, {score}'

      else:
        color = (255, 255, 0)
        text = f'Mask Incorrect, {score}'

      return color, text

time: 1.47 ms (started: 2024-04-04 11:05:34 +00:00)


In [ ]:
def _get_model_size(path):
    """
      Get the size of the model file
    """

    # Read and calculate model size
    model_path = f"{MODEL_PATH}/{path}"

    if os.path.isdir(model_path):
        model_path = f'{model_path}/model.pkl'

    print(f"Model Size: {os.path.getsize(model_path)} byte")

In [ ]:
def _esrgan(imgs):
    """
      Enhances the input low-resolution image using ESRGAN (Enhanced Super-Resolution Generative Adversarial Network)

      Code derived from Tensorflow Hub tutorial,
      https://www.tensorflow.org/hub/tutorials/image_enhancing, demonstrating ESRGAN obtained by by Xintao Wang et.al.
    """

    # Load the ESRGAN model from TensorFlow Hub
    esrgan_path = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
    esrgan = hub.load(esrgan_path)

    hr_imgs = []

    for img in imgs:

        # Convert the input image to float32
        img = tf.cast(img, tf.float32)

        # Crop the input image to a size that is a multiple of 4
        target_height = (img.shape[0] // 4) * 4
        target_width = (img.shape[1] // 4) * 4
        lr = tf.image.crop_to_bounding_box(img, 0, 0, target_height, target_width)

        # Convert the cropped image to a TensorFlow tensor
        lr = tf.convert_to_tensor(lr, dtype=tf.float32)
        lr = tf.expand_dims(lr, 0)

        # Apply ESRGAN to the low-resolution image to obtain the high-resolution image
        hr = esrgan(lr)
        hr_img = tf.squeeze(hr)

        # Normalize the pixel values of the high-resolution image to the range [0, 255]
        hr_img = (hr_img - np.min(hr_img)) / (np.max(hr_img) - np.min(hr_img)) * 255
        hr_img = np.clip(hr_img, 0, 255).astype(np.uint8)

        hr_imgs.append(hr_img)

    return hr_imgs

In [ ]:
def _gradcam_heatmap(img, model, last_conv):
    """
    Compute the Grad-CAM heatmap for a given input image array
    Code obtained from Keras example by fchollet
        https://keras.io/examples/vision/grad_cam/
    """

    # Create a model to get the activations of the last conv layer and predictions
    model = Model(model.inputs, [model.get_layer(last_conv).output, model.output])

    # Compute the gradient of the top predicted class for the input image
    with tf.GradientTape() as tape:
        output, preds = model(img)
        pred_label = tf.argmax(preds[0])
        class_channel = preds[:, pred_label]

    # Compute the gradient of the output neuron with respect to the last conv layer
    grads = tape.gradient(class_channel, output)

    # Compute mean intensity of the gradient over each feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel by importance w.r.t. predicted class and sum for heatmap
    heatmap = output[0] @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize heatmap for visualization
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    return heatmap.numpy(), pred_label.numpy()

time: 1.56 ms (started: 2024-04-19 12:34:27 +00:00)


In [ ]:
def _gradcam(X, y, input_size, preprocess, last_conv, model_path):
    """
      Generate Grad-CAM visualization from trained model for a given input image
      Code obtained from Keras example by fchollet
        https://keras.io/examples/vision/grad_cam/
    """

    # Prepare image
    img = (_resize(X, input_size))[0]
    img = np.expand_dims(img, axis=0)

    if preprocess:
      img = preprocess(img)

    # Load trained model
    model = load_model(f"{MODEL_PATH}/{model_path}")

    # Remove last layer's activation
    model.layers[-1].activation = None

    # Generate class activation heatmap using Grad-CAM technique
    heatmap, label = _gradcam_heatmap(img, model, last_conv)

    # Convert the original image to a  array
    img = keras.utils.img_to_array(X[0])

    # Rescale the heatmap to a range of 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = matplotlib.colormaps["jet"]

    # Extract the RGB values from the jet colormap
    colors = (jet(np.arange(256))[:, :3])[heatmap]

    # Convert the colorized heatmap to an image
    colorized_heatmap = keras.utils.array_to_img(colors)
    colorized_heatmap = colorized_heatmap.resize((img.shape[1], img.shape[0]))
    colorized_heatmap = keras.utils.img_to_array(colorized_heatmap)

    # Superimpose the heatmap on the original image
    img = colorized_heatmap * 0.5 + img
    img = keras.utils.array_to_img(img)

    return img, label

time: 2.48 ms (started: 2024-04-19 12:34:27 +00:00)


In [ ]:
def _plot_gradcam(imgs, labels, input_size=(224,224), preprocess=None, last_conv=None, model_path=None):
    """
      Plot images along with corresponding Grad-CAM heatmaps
    """

    fig, axes = plt.subplots(1, 3, figsize=(8, 12))

    for i, (X, y) in enumerate(zip(imgs, labels)):
        img, y_pred = _gradcam([X], y, input_size, preprocess, last_conv, model_path)


        # Plot image
        ax = axes[i]
        ax.imshow(img)
        ax.set_title(f"True: {labels[i]}, Predicted: {y_pred}")
        ax.axis('off')

    # Show the plot
    plt.tight_layout()
    plt.show()

### Main Functions

In [ ]:
def MaskDetection(path_to_testset, model_type, n=4, model_name=None):
    """
      Perform mask detection using the specified model and preprocessing methods

      Params:
        - path_to_testset: Dataset path to be evaluated
        - model_type: Type of model used for detection
        - n: Number of test samples to evaluate or 'All' for all samples
        - model_name: Specific model name to be evaluated, otherwise default models for each algorithm will be loaded

      Returns:
        - None
    """


    # Call different functions, and load models
    if model_type == 'ResNet50':
          model_path = f"{MODEL_PATH}/Pretrained-CNN/{'20240410-173718.h5' if not model_name else model_name}"
          func = pretrainedCNN(preprocess=resnet50_preprocess)
          model = load_model(model_path)

    elif model_type == 'VGG16':
          model_path = f"{MODEL_PATH}/Pretrained-CNN/{'20240405-163933.h5' if not model_name else model_name}"
          func = pretrainedCNN(preprocess=vgg16_preprocess)
          model = load_model(model_path)

    elif model_type == 'MobileNet':
          model_path = f"{MODEL_PATH}/Pretrained-CNN/{'20240405-093232.h5' if not model_name else model_name}"
          func = pretrainedCNN(preprocess=mobilenet_preprocess)
          model = load_model(model_path)

    elif model_type == 'EfficientNet':
          model_path = f"{MODEL_PATH}/Pretrained-CNN/{'20240410-170150.h5' if not model_name else model_name}"
          func = pretrainedCNN(preprocess=efficientnet_preproces)
          model = load_model(model_path)

    elif model_type == 'CustomCNN':
          model_path = f"{MODEL_PATH}/Custom-CNN/{'20240409-183732.h5' if not model_name else model_name}"
          func = customCNN()
          model = load_model(model_path)

    elif model_type == 'SVM-HOG':
          model_path = f"{MODEL_PATH}/SVM-HOG/{'20240404-233425.pkl' if not model_name else model_name}"
          func = SVMWithHOG()
          model = joblib.load(model_path)

    elif model_type == 'SVM-SIFT':
          model_fn = f"SVM-SIFT/{'20240404-231342' if not model_name else model_name}"
          model_path = f"{MODEL_PATH}/{model_fn}"
          func = SVMWithSIFT(filepath=model_fn)
          # For SVM with SIFT, model is stored inside directory with k-means model
          model = joblib.load(f'{model_path}/model.pkl')

    # Evaluate entire test set
    if n == 'All':
      func._evaluate_test_set(model)

    # Otherwise, evaluate only n images (default is 4)
    else:
      # Import test data
      X, y = _import_data(path_to_testset, n=n)

      # Get predicted labels
      y, y_pred, _ = func._evaluate_frame(X, y, model)
      _plot_images(X, y, y_pred)

In [ ]:
def MaskDetectionVideo(model_path='Pretrained-CNN/20240410-173718.h5', model_type='ResNet50'):
    """
      Detects masks in a video using the specified model

      Params:
        - model_path: The path to the model, default: 'ResNet50'
        - model_type: The type of model to be used, default: 'ResNet50'

      Returns:
        - Animation: Animation object showing the processed video with mask detection
    """

    # Load model
    model = load_model(f"{MODEL_PATH}/{model_path}")
    print('Model Loaded successcfully!')

    # Call different functions depends on models
    if model_type == 'ResNet50':
          func = pretrainedCNN(preprocess=resnet50_preprocess)

    elif model_type == 'VGG16':
          func = pretrainedCNN(preprocess=vgg16_preprocess)

    elif model_type == 'MobileNet':
          func = pretrainedCNN(preprocess=mobilenet_preprocess)

    elif model_type == 'EfficientNet':
          func = pretrainedCNN(preprocess=efficientnet_preproces)

    elif model_type == 'CustomCNN':
          func = customCNN()

    elif model_type == 'SVM-HOG':
          func = SVMWithHOG()

    elif model_type == 'SVM-SIFT':
          func = SVMWithSIFT()

    # Perform face mask detector using specified model
    anim = face_detector('Video1.mp4', yunet_face_detector, func, model)
    print('Precessing video complete!')

    return anim